**Thành Viên: Phạm Đức Thể, Trần Triệu Vũ, Võ Minh Trí**

**MSSV: 19522253, 19522539, 19522396**

**Lớp: DS200.M21**

**Nội Dung: [PySpark - DataFrame](https://drive.google.com/file/d/1Pj0FHYEAzyOHZ5HB8beJL_6N6SQALgvY/view?usp=sharing)**

**Ngày: 04/05/2022** 

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/15461DibXLPMiXKt8tFU3a2YAUDWNKSKs?usp=sharing)

# **Big Data - PySpark - DataFrame**

In [ ]:
!pip install pyspark==3.0.1

     |████████████████████████████████| 204.2 MB 34 kB/s 
     |████████████████████████████████| 198 kB 50.2 MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612243 sha256=db700849fa2acc9db466f80f399dbebc7dedaa1032900cb5958de4bbe1c66a81
  Stored in directory: /root/.cache/pip/wheels/5e/34/fa/b37b5cef503fc5148b478b2495043ba61b079120b7ff379f9b
Successfully built pyspark


In [ ]:
import pyspark
from pyspark.sql import SparkSession
from heapq import nlargest, nsmallest
from pyspark.sql import functions as f
from pyspark.sql.window import Window

In [ ]:
spark = SparkSession.builder.appName("Assignment04").getOrCreate()

In [ ]:
PATH = '/content/drive/MyDrive/TÀI LIỆU HỌC TẬP ĐẠI HỌC 2019-2023/NĂM 3 2021-2022/HỌC KỲ 2 2021 - 2022/DS200.M21 - PHÂN TÍCH DỮ LIỆU LỚN/THỰC HÀNH/LAB/Assignment04 - 04 05 2022/'

In [ ]:
movies = spark.read.csv(path=PATH+'movies_small.csv', 
                        sep=',', header=True, quote='"', 
                        schema='movieId INT, title STRING, genres STRING')

movies.printSchema()
movies.show(truncate=False)

root
 |-- movieId: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- genres: string (nullable = true)

+-------+-------------------------------------+-------------------------------------------+
|movieId|title                                |genres                                     |
+-------+-------------------------------------+-------------------------------------------+
|1      |Toy Story (1995)                     |Adventure|Animation|Children|Comedy|Fantasy|
|2      |Jumanji (1995)                       |Adventure|Children|Fantasy                 |
|3      |Grumpier Old Men (1995)              |Comedy|Romance                             |
|4      |Waiting to Exhale (1995)             |Comedy|Drama|Romance                       |
|5      |Father of the Bride Part II (1995)   |Comedy                                     |
|6      |Heat (1995)                          |Action|Crime|Thriller                      |
|7      |Sabrina (1995)                       |Come

In [ ]:
ratings = spark.read.csv(path=PATH+'ratings_small.csv', 
                         sep=',', header=True, quote='"', 
                         schema='userId INT, movieId INT, rating DOUBLE, timestamp INT') \
                         .withColumnRenamed('timestamp', 'timestamp_unix') \
                         .withColumn('timestamp', f.to_timestamp(f.from_unixtime('timestamp_unix')))

ratings.printSchema()
ratings.show()

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- timestamp_unix: integer (nullable = true)
 |-- timestamp: timestamp (nullable = true)

+------+-------+------+--------------+-------------------+
|userId|movieId|rating|timestamp_unix|          timestamp|
+------+-------+------+--------------+-------------------+
|     1|      1|   4.0|     964982703|2000-07-30 18:45:03|
|     1|      3|   4.0|     964981247|2000-07-30 18:20:47|
|     1|      6|   4.0|     964982224|2000-07-30 18:37:04|
|     1|     47|   5.0|     964983815|2000-07-30 19:03:35|
|     1|     50|   5.0|     964982931|2000-07-30 18:48:51|
|     1|     70|   3.0|     964982400|2000-07-30 18:40:00|
|     1|    101|   5.0|     964980868|2000-07-30 18:14:28|
|     1|    110|   4.0|     964982176|2000-07-30 18:36:16|
|     1|    151|   5.0|     964984041|2000-07-30 19:07:21|
|     1|    157|   5.0|     964984100|2000-07-30 19:08:20|
|     1|    163|   

In [ ]:
movies_ratings = movies.join(ratings, ['movieId'])
movies_ratings.show(truncate=False)

+-------+-----------------------------------------+-------------------------------------------+------+------+--------------+-------------------+
|movieId|title                                    |genres                                     |userId|rating|timestamp_unix|timestamp          |
+-------+-----------------------------------------+-------------------------------------------+------+------+--------------+-------------------+
|1      |Toy Story (1995)                         |Adventure|Animation|Children|Comedy|Fantasy|1     |4.0   |964982703     |2000-07-30 18:45:03|
|3      |Grumpier Old Men (1995)                  |Comedy|Romance                             |1     |4.0   |964981247     |2000-07-30 18:20:47|
|6      |Heat (1995)                              |Action|Crime|Thriller                      |1     |4.0   |964982224     |2000-07-30 18:37:04|
|47     |Seven (a.k.a. Se7en) (1995)              |Mystery|Thriller                           |1     |5.0   |964983815     |2000-0

## **Câu 1:** Show the number of movies made in each year. The results are sorted by year.

In [ ]:
movies.withColumn('year', f.regexp_extract(f.col('title'), '\(([^()]+)\)$', 1).cast('int')) \
.groupBy('year').count().sort('year').show(100,truncate=False)

+----+-----+
|year|count|
+----+-----+
|null|25   |
|1902|1    |
|1903|1    |
|1908|1    |
|1915|1    |
|1916|4    |
|1917|1    |
|1919|1    |
|1920|2    |
|1921|1    |
|1922|1    |
|1923|4    |
|1924|5    |
|1925|4    |
|1926|5    |
|1927|7    |
|1928|4    |
|1929|4    |
|1930|5    |
|1931|14   |
|1932|9    |
|1933|12   |
|1934|11   |
|1935|13   |
|1936|18   |
|1937|16   |
|1938|15   |
|1939|23   |
|1940|25   |
|1941|18   |
|1942|23   |
|1943|10   |
|1944|16   |
|1945|17   |
|1946|23   |
|1947|20   |
|1948|20   |
|1949|25   |
|1950|21   |
|1951|22   |
|1952|16   |
|1953|30   |
|1954|23   |
|1955|36   |
|1956|30   |
|1957|33   |
|1958|31   |
|1959|37   |
|1960|37   |
|1961|34   |
|1962|40   |
|1963|39   |
|1964|43   |
|1965|47   |
|1966|42   |
|1967|42   |
|1968|42   |
|1969|35   |
|1970|33   |
|1971|47   |
|1972|39   |
|1973|58   |
|1974|45   |
|1975|42   |
|1976|44   |
|1977|63   |
|1978|59   |
|1979|69   |
|1980|89   |
|1981|92   |
|1982|87   |
|1983|83   |
|1984|101  |
|1985|126  |

## **Câu 2:** Show the number of movies belonging to each genre made in each year. The results are sorted by year. The results of each genre are supposed to be displayed in a column.

In [ ]:
movies_2 = movies.withColumn('genres', f.explode(f.split(f.col('genres'), '\|'))) \
.withColumn('year', f.regexp_extract(f.col('title'), '\(([^()]+)\)$', 1).cast('int')) \
.groupBy(['genres', 'year']).count().sort('year').select('genres', 'count', 'year')
movies_2.show(100,truncate=False)

+------------------+-----+----+
|genres            |count|year|
+------------------+-----+----+
|Comedy            |3    |null|
|(no genres listed)|9    |null|
|Fantasy           |2    |null|
|Drama             |6    |null|
|Crime             |2    |null|
|Sci-Fi            |5    |null|
|War               |1    |null|
|Animation         |5    |null|
|Adventure         |3    |null|
|Horror            |3    |null|
|Children          |1    |null|
|Action            |7    |null|
|Thriller          |3    |null|
|Adventure         |1    |1902|
|Action            |1    |1902|
|Fantasy           |1    |1902|
|Sci-Fi            |1    |1902|
|Crime             |1    |1903|
|Western           |1    |1903|
|Comedy            |1    |1908|
|Sci-Fi            |1    |1908|
|Animation         |1    |1908|
|Drama             |1    |1915|
|War               |1    |1915|
|Comedy            |1    |1916|
|Adventure         |1    |1916|
|Sci-Fi            |1    |1916|
|Fantasy           |1    |1916|
|Romance

In [ ]:
movies_2.groupby('year').pivot('genres').sum('count').fillna(0).sort('year').show(100, truncate=False)

+----+------------------+------+---------+---------+--------+------+-----+-----------+-----+-------+---------+------+----+-------+-------+-------+------+--------+---+-------+
|year|(no genres listed)|Action|Adventure|Animation|Children|Comedy|Crime|Documentary|Drama|Fantasy|Film-Noir|Horror|IMAX|Musical|Mystery|Romance|Sci-Fi|Thriller|War|Western|
+----+------------------+------+---------+---------+--------+------+-----+-----------+-----+-------+---------+------+----+-------+-------+-------+------+--------+---+-------+
|0   |9                 |7     |3        |5        |1       |3     |2    |0          |6    |2      |0        |3     |0   |0      |0      |0      |5     |3       |1  |0      |
|1902|0                 |1     |1        |0        |0       |0     |0    |0          |0    |1      |0        |0     |0   |0      |0      |0      |1     |0       |0  |0      |
|1903|0                 |0     |0        |0        |0       |0     |1    |0          |0    |0      |0        |0     |0   |0  

## **Câu 3:** For each userID, show the average rating of that user for each genre. The results are sorted by userID. The results of each genre are displayed in a separate column.

In [ ]:
df1 = ratings.select('userId', 'movieId', 'rating')
df1.show(100,truncate=False)

+------+-------+------+
|userId|movieId|rating|
+------+-------+------+
|1     |1      |4.0   |
|1     |3      |4.0   |
|1     |6      |4.0   |
|1     |47     |5.0   |
|1     |50     |5.0   |
|1     |70     |3.0   |
|1     |101    |5.0   |
|1     |110    |4.0   |
|1     |151    |5.0   |
|1     |157    |5.0   |
|1     |163    |5.0   |
|1     |216    |5.0   |
|1     |223    |3.0   |
|1     |231    |5.0   |
|1     |235    |4.0   |
|1     |260    |5.0   |
|1     |296    |3.0   |
|1     |316    |3.0   |
|1     |333    |5.0   |
|1     |349    |4.0   |
|1     |356    |4.0   |
|1     |362    |5.0   |
|1     |367    |4.0   |
|1     |423    |3.0   |
|1     |441    |4.0   |
|1     |457    |5.0   |
|1     |480    |4.0   |
|1     |500    |3.0   |
|1     |527    |5.0   |
|1     |543    |4.0   |
|1     |552    |4.0   |
|1     |553    |5.0   |
|1     |590    |4.0   |
|1     |592    |4.0   |
|1     |593    |4.0   |
|1     |596    |5.0   |
|1     |608    |5.0   |
|1     |648    |3.0   |
|1     |661    |

In [ ]:
df2 = movies.select('movieId', 'genres').withColumn('genres_array', f.split('genres', '[|]')) \
.withColumn('genre', f.explode('genres_array')).select('movieId', 'genre')
df2.show(100,truncate=False)

+-------+---------+
|movieId|genre    |
+-------+---------+
|1      |Adventure|
|1      |Animation|
|1      |Children |
|1      |Comedy   |
|1      |Fantasy  |
|2      |Adventure|
|2      |Children |
|2      |Fantasy  |
|3      |Comedy   |
|3      |Romance  |
|4      |Comedy   |
|4      |Drama    |
|4      |Romance  |
|5      |Comedy   |
|6      |Action   |
|6      |Crime    |
|6      |Thriller |
|7      |Comedy   |
|7      |Romance  |
|8      |Adventure|
|8      |Children |
|9      |Action   |
|10     |Action   |
|10     |Adventure|
|10     |Thriller |
|11     |Comedy   |
|11     |Drama    |
|11     |Romance  |
|12     |Comedy   |
|12     |Horror   |
|13     |Adventure|
|13     |Animation|
|13     |Children |
|14     |Drama    |
|15     |Action   |
|15     |Adventure|
|15     |Romance  |
|16     |Crime    |
|16     |Drama    |
|17     |Drama    |
|17     |Romance  |
|18     |Comedy   |
|19     |Comedy   |
|20     |Action   |
|20     |Comedy   |
|20     |Crime    |
|20     |Drama    |


In [ ]:
df3 = df2.join(df1, on = 'movieId', how = 'inner')
df3.show(100,truncate=False)

+-------+---------+------+------+
|movieId|genre    |userId|rating|
+-------+---------+------+------+
|1      |Fantasy  |1     |4.0   |
|1      |Comedy   |1     |4.0   |
|1      |Children |1     |4.0   |
|1      |Animation|1     |4.0   |
|1      |Adventure|1     |4.0   |
|3      |Romance  |1     |4.0   |
|3      |Comedy   |1     |4.0   |
|6      |Thriller |1     |4.0   |
|6      |Crime    |1     |4.0   |
|6      |Action   |1     |4.0   |
|47     |Thriller |1     |5.0   |
|47     |Mystery  |1     |5.0   |
|50     |Thriller |1     |5.0   |
|50     |Mystery  |1     |5.0   |
|50     |Crime    |1     |5.0   |
|70     |Thriller |1     |3.0   |
|70     |Horror   |1     |3.0   |
|70     |Comedy   |1     |3.0   |
|70     |Action   |1     |3.0   |
|101    |Romance  |1     |5.0   |
|101    |Crime    |1     |5.0   |
|101    |Comedy   |1     |5.0   |
|101    |Adventure|1     |5.0   |
|110    |War      |1     |4.0   |
|110    |Drama    |1     |4.0   |
|110    |Action   |1     |4.0   |
|151    |War  

In [ ]:
df3 = df3.withColumn('rating', f.col('rating').cast('float'))
df3 = df3.withColumn('userId', f.col('userId').cast('int'))
df3.groupBy('userId').pivot('genre').avg('rating').fillna(0).sort('userId').show(100,truncate=10)

+------+------------------+----------+----------+----------+----------+----------+----------+-----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+
|userId|(no genres listed)|    Action| Adventure| Animation|  Children|    Comedy|     Crime|Documentary|     Drama|   Fantasy| Film-Noir|    Horror|      IMAX|   Musical|   Mystery|   Romance|    Sci-Fi|  Thriller|       War|   Western|
+------+------------------+----------+----------+----------+----------+----------+----------+-----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+
|     1|               0.0|4.32222...|4.38823...|4.68965...|4.54761...|4.27710...|4.35555...|        0.0|4.52941...|4.29787...|       5.0|3.47058...|       0.0|4.68181...|4.16666...|4.30769...|     4.225|4.14545...|       4.5|4.28571...|
|     2|               0.0|3.95454...|4.16666...

## **Câu 4:** Show the name, the year, the number of ratings, and the average rating of each movie. The results are sorted by the years and (then) the names of the movies.

In [ ]:
movie1 = movies.select('movieId', 'title').withColumn('year', f.substring('title',-5,4))
movie1 = movie1.withColumn('sub1', f.split('title', ' ')).withColumn('sub2', f.element_at('sub1', -1)) \
.withColumn('sub2', f.split('sub2', '123456')).withColumn('sub3', f.array_except('sub1', 'sub2')) \
.withColumn('name', f.concat_ws(' ', f.col('sub3')))
movie1.show(100,truncate=False)

+-------+-------------------------------------------------------------------------------+----+-----------------------------------------------------------------------------------------------+--------+---------------------------------------------------------------------------------------+------------------------------------------------------------------------+
|movieId|title                                                                          |year|sub1                                                                                           |sub2    |sub3                                                                                   |name                                                                    |
+-------+-------------------------------------------------------------------------------+----+-----------------------------------------------------------------------------------------------+--------+-------------------------------------------------------------------------------

In [ ]:
ratings = ratings.withColumn('rating', f.col('rating').cast('float'))
ratings = ratings.withColumn('movieId', f.col('movieId').cast('int'))
rate1 = ratings.groupBy('movieId').avg('rating').sort('movieId')
rate1.show(100,truncate=False)

+-------+------------------+
|movieId|avg(rating)       |
+-------+------------------+
|1      |3.9209302325581397|
|2      |3.4318181818181817|
|3      |3.2596153846153846|
|4      |2.357142857142857 |
|5      |3.0714285714285716|
|6      |3.946078431372549 |
|7      |3.185185185185185 |
|8      |2.875             |
|9      |3.125             |
|10     |3.496212121212121 |
|11     |3.6714285714285713|
|12     |2.4210526315789473|
|13     |3.125             |
|14     |3.8333333333333335|
|15     |3.0               |
|16     |3.926829268292683 |
|17     |3.7761194029850746|
|18     |3.7               |
|19     |2.727272727272727 |
|20     |2.5               |
|21     |3.49438202247191  |
|22     |3.2222222222222223|
|23     |3.125             |
|24     |3.125             |
|25     |3.625             |
|26     |3.5               |
|27     |3.3333333333333335|
|28     |4.2272727272727275|
|29     |4.0131578947368425|
|30     |3.0               |
|31     |3.1842105263157894|
|32     |3.983

In [ ]:
rate2 = ratings.groupBy('movieId').count().sort('movieId')
rate2.show(100,truncate=False)

+-------+-----+
|movieId|count|
+-------+-----+
|1      |215  |
|2      |110  |
|3      |52   |
|4      |7    |
|5      |49   |
|6      |102  |
|7      |54   |
|8      |8    |
|9      |16   |
|10     |132  |
|11     |70   |
|12     |19   |
|13     |8    |
|14     |18   |
|15     |13   |
|16     |82   |
|17     |67   |
|18     |20   |
|19     |88   |
|20     |15   |
|21     |89   |
|22     |36   |
|23     |16   |
|24     |28   |
|25     |76   |
|26     |13   |
|27     |9    |
|28     |11   |
|29     |38   |
|30     |3    |
|31     |38   |
|32     |177  |
|34     |128  |
|36     |67   |
|38     |4    |
|39     |104  |
|40     |2    |
|41     |15   |
|42     |7    |
|43     |8    |
|44     |46   |
|45     |32   |
|46     |15   |
|47     |203  |
|48     |68   |
|49     |1    |
|50     |204  |
|52     |28   |
|53     |2    |
|54     |3    |
|55     |1    |
|57     |8    |
|58     |37   |
|60     |34   |
|61     |6    |
|62     |80   |
|63     |7    |
|64     |5    |
|65     |31   |
|66     

In [ ]:
rate3 = rate2.join(rate1, on = 'movieId', how = 'inner')
rate3.show(100,truncate=False)

+-------+-----+------------------+
|movieId|count|avg(rating)       |
+-------+-----+------------------+
|1580   |165  |3.487878787878788 |
|2366   |25   |3.64              |
|3175   |75   |3.58              |
|1088   |42   |3.369047619047619 |
|32460  |4    |4.25              |
|44022  |23   |3.217391304347826 |
|96488  |4    |4.25              |
|1238   |9    |4.055555555555555 |
|1342   |11   |2.5               |
|1591   |26   |2.6346153846153846|
|1645   |51   |3.411764705882353 |
|4519   |9    |3.3333333333333335|
|2142   |10   |2.7               |
|471    |40   |3.55              |
|3997   |12   |1.8333333333333333|
|833    |6    |2.0               |
|3918   |9    |3.2777777777777777|
|7982   |4    |3.25              |
|1959   |15   |3.6666666666666665|
|68135  |10   |3.55              |
|5300   |1    |4.0               |
|2659   |1    |2.0               |
|5803   |7    |2.5               |
|76143  |1    |4.5               |
|140541 |1    |4.0               |
|2122   |11   |2.454

In [ ]:
movie1 = movie1.withColumn('year', f.col('year').cast('int'))
result = rate3.join(movie1, on = 'movieId', how = 'inner').select('name', 'year', 'count', 'avg(rating)') \
.sort('name').sort('year')
result.show(100,truncate=False)

+----------------------------------------------------------+----+-----+------------------+
|name                                                      |year|count|avg(rating)       |
+----------------------------------------------------------+----+-----+------------------+
|Nocturnal                                                 |null|1    |3.0               |
|Pocahontas II: Journey to a New World (1998)              |null|1    |2.0               |
|Black                                                     |null|1    |5.0               |
|Justice League: Doom (2012)                               |null|1    |5.0               |
|3 dev adam (Three Giant Men) (1973)                       |null|1    |0.5               |
|The Adventures of Sherlock Holmes and Doctor              |null|1    |5.0               |
|                                                          |null|2    |4.5               |
|Maria Bamford: Old                                        |null|1    |1.0               |

## **Câu 5:** For each user ID, show the genres that received highest rating from that user and the list of top 5 rated movies belong to that genres that hasn’t been rated by that user. The results are sorted by the user ID.

tìm tất cả các thể loại phim

In [ ]:
type_genres = movies.withColumn('genres', f.split('genres','[|]')).withColumn('genres', f.explode('genres'))\
.withColumn('temp', f.lit('temp_')).groupBy('temp').agg(f.collect_set('genres'))

In [ ]:
type_genres.show()

+-----+--------------------+
| temp| collect_set(genres)|
+-----+--------------------+
|temp_|[Thriller, Action...|
+-----+--------------------+



tìm tất cả các bộ phim

In [ ]:
type_title = movies.withColumn('genres', f.split('genres','[|]')).withColumn('genres', f.explode('genres'))\
.withColumn('temp', f.lit('temp_')).groupBy('temp').agg(f.collect_set('title'))

In [ ]:
type_title.show()

+-----+--------------------+
| temp|  collect_set(title)|
+-----+--------------------+
|temp_|[Airport '77 (197...|
+-----+--------------------+



In [ ]:
userid_genres = movies.join(ratings, on=['movieId'], how='inner').withColumn('genres', f.split('genres','[|]')) \
.withColumn('genres', f.explode('genres')).groupBy('userId','genres').sum('rating').sort('userId')

In [ ]:
userid_genres.show()

+------+---------+-----------+
|userId|   genres|sum(rating)|
+------+---------+-----------+
|     1|Animation|      136.0|
|     1|    Crime|      196.0|
|     1|Film-Noir|        5.0|
|     1|      War|       99.0|
|     1| Children|      191.0|
|     1|  Romance|      112.0|
|     1|  Western|       30.0|
|     1|  Mystery|       75.0|
|     1|   Comedy|      355.0|
|     1|  Musical|      103.0|
|     1|    Drama|      308.0|
|     1|  Fantasy|      202.0|
|     1|   Horror|       59.0|
|     1|Adventure|      373.0|
|     1|   Sci-Fi|      169.0|
|     1|   Action|      389.0|
|     1| Thriller|      228.0|
|     2|   Horror|        3.0|
|     2|   Action|       43.5|
|     2|  Mystery|        8.0|
+------+---------+-----------+
only showing top 20 rows



In [ ]:
userid_title = movies.join(ratings, on=['movieId'], how='inner').withColumn('genres', f.split('genres','[|]'))\
.groupBy('title').sum('rating')

In [ ]:
userid_title.show()

+--------------------+-----------+
|               title|sum(rating)|
+--------------------+-----------+
|       Psycho (1960)|      335.0|
|Men in Black (a.k...|      575.5|
|Gulliver's Travel...|        9.0|
|Heavenly Creature...|       82.5|
|    Elizabeth (1998)|       84.5|
|Before Night Fall...|       21.5|
|O Brother, Where ...|      358.0|
|Snow White and th...|      278.5|
| Three Wishes (1995)|        3.0|
|When We Were King...|       39.0|
|   Annie Hall (1977)|      224.5|
| If Lucy Fell (1996)|        5.0|
|First Blood (Ramb...|      106.5|
|Don't Tell Mom th...|       30.5|
| Nut Job, The (2014)|       13.0|
|22 Jump Street (2...|       70.0|
|   Deadpool 2 (2018)|       46.5|
|Starship Troopers...|        3.0|
|Voices from the L...|       21.5|
|Night of the Livi...|      105.0|
+--------------------+-----------+
only showing top 20 rows



In [ ]:
max_rating = movies.join(ratings, on=['movieId'], how='inner').withColumn('genres', f.split('genres','[|]')) \
.withColumn('genres', f.explode('genres')).groupBy('userId','genres').sum('rating').sort('userId') \
.groupBy('userId').agg(f.max('sum(rating)').alias('sum(rating)'))

In [ ]:
max_rating.sort('userId').show()

+------+-----------+
|userId|sum(rating)|
+------+-----------+
|     1|      389.0|
|     2|       66.0|
|     3|       63.0|
|     4|      418.0|
|     5|       95.0|
|     6|      506.0|
|     7|      208.5|
|     8|       77.0|
|     9|       72.0|
|    10|      260.0|
|    11|      163.0|
|    12|       96.0|
|    13|       56.0|
|    14|       89.0|
|    15|      211.5|
|    16|      179.0|
|    17|      213.5|
|    18|      849.0|
|    19|      998.0|
|    20|      434.0|
+------+-----------+
only showing top 20 rows



tên thể loại được người dùng vote nhiều nhất

In [ ]:
genres_max = max_rating.join(userid_genres, on=['sum(rating)','userId'], how='inner').sort('userId').select(f.col('userId'), f.col('genres'))

In [ ]:
genres_max.show()

+------+--------+
|userId|  genres|
+------+--------+
|     1|  Action|
|     2|   Drama|
|     3|  Sci-Fi|
|     4|   Drama|
|     5|   Drama|
|     6|   Drama|
|     7|  Action|
|     8|  Comedy|
|     9|   Drama|
|    10| Romance|
|    11|  Action|
|    12| Romance|
|    13|   Drama|
|    14|   Drama|
|    15|  Sci-Fi|
|    16|   Drama|
|    17|   Drama|
|    18|   Drama|
|    19|  Comedy|
|    20|Children|
+------+--------+
only showing top 20 rows



Tìm các bộ phim theo rating (phim được user rating)

In [ ]:
title_rated = movies.join(ratings, on=['movieId'], how='inner').withColumn('genres', f.split('genres','[|]'))\
.groupBy('userId').agg(f.collect_list('title').alias('title(rated)')).sort('userId')
#.groupBy('userId').agg(f.max('sum(rating)').alias('max of sum(rating)')).sort('userId').show()

In [ ]:
title_rated.show(truncate=100)

+------+----------------------------------------------------------------------------------------------------+
|userId|                                                                                        title(rated)|
+------+----------------------------------------------------------------------------------------------------+
|     1|[Toy Story (1995), Grumpier Old Men (1995), Heat (1995), Seven (a.k.a. Se7en) (1995), Usual Suspe...|
|     2|[Shawshank Redemption, The (1994), Tommy Boy (1995), Good Will Hunting (1997), Gladiator (2000), ...|
|     3|[Dangerous Minds (1995), Schindler's List (1993), Courage Under Fire (1996), Operation Dumbo Drop...|
|     4|[Get Shorty (1995), Twelve Monkeys (a.k.a. 12 Monkeys) (1995), To Die For (1995), Seven (a.k.a. S...|
|     5|[Toy Story (1995), Get Shorty (1995), Babe (1995), Dead Man Walking (1995), Clueless (1995), Usua...|
|     6|[Jumanji (1995), Grumpier Old Men (1995), Waiting to Exhale (1995), Father of the Bride Part II (...|
|     7|[T

tìm tất cả phim theo từng genres rating cao nhất

In [ ]:
title_all = movies.join(ratings, on=['movieId'], how='inner').withColumn('genres', f.split('genres','[|]'))\
.withColumn('genres', f.explode('genres')).groupBy('genres').agg(f.collect_set('title').alias('title all of'))

In [ ]:
title_all.show(truncate=100)

+------------------+----------------------------------------------------------------------------------------------------+
|            genres|                                                                                        title all of|
+------------------+----------------------------------------------------------------------------------------------------+
|             Crime|[Stealing Rembrandt (Rembrandt) (2003), Girl with the Dragon Tattoo, The (Män som hatar kvinnor) ...|
|           Romance|[Vampire in Brooklyn (1995), Hysteria (2011), Far from Heaven (2002), Honey (2003), Ghost Town (2...|
|          Thriller|[Element of Crime, The (Forbrydelsens Element) (1984), 'Salem's Lot (2004), Transformers (2007), ...|
|         Adventure|[Ice Age: Collision Course (2016), Masters of the Universe (1987), Terminator Genisys (2015), Tit...|
|             Drama|[Airport '77 (1977), Meet John Doe (1941), 'Salem's Lot (2004), Element of Crime, The (Forbrydels...|
|               War|[Gen

join bảng

In [ ]:
table = title_rated.join(genres_max, on=['userId'], how='inner').join(title_all, on=['genres'], how='inner')\
.withColumn('title', f.array_except('title all of','title(rated)'))\
.withColumn('title', f.explode('title')).sort('userId')\
.select(f.col('genres'), f.col('userId'), f.col('title'))
#.join(userid_title, on=['userId', 'title'], how='inner')

In [ ]:
table.show()

+------+------+--------------------+
|genres|userId|               title|
+------+------+--------------------+
|Action|     1|Kingsman: The Sec...|
|Action|     1|Throne of Blood (...|
|Action|     1|   Alex Cross (2012)|
|Action|     1| Chill Factor (1999)|
|Action|     1|Kiss of the Drago...|
|Action|     1|     Snatched (2017)|
|Action|     1|     Red Heat (1988)|
|Action|     1|Masters of the Un...|
|Action|     1| Iron Soldier (2010)|
|Action|     1|21 Jump Street (2...|
|Action|     1|Animatrix, The (2...|
|Action|     1|        Steel (1997)|
|Action|     1|        Blitz (2011)|
|Action|     1|Big Boss, The (Fi...|
|Action|     1|  Hunted, The (1995)|
|Action|     1|Stealing Rembrand...|
|Action|     1|Mechanic, The (2011)|
|Action|     1|Terminator Genisy...|
|Action|     1|   Titan A.E. (2000)|
|Action|     1| Transformers (2007)|
+------+------+--------------------+
only showing top 20 rows



In [ ]:
final_5 = table.join(userid_title, on=['title'], how='inner')

In [ ]:
final_5.show()

+--------------------+------+------+-----------+
|               title|genres|userId|sum(rating)|
+--------------------+------+------+-----------+
|22 Jump Street (2...| Crime|    55|       70.0|
|22 Jump Street (2...| Crime|   433|       70.0|
|22 Jump Street (2...|Comedy|   471|       70.0|
|22 Jump Street (2...|Comedy|    31|       70.0|
|22 Jump Street (2...|Comedy|   516|       70.0|
|22 Jump Street (2...|Comedy|   255|       70.0|
|22 Jump Street (2...|Comedy|   321|       70.0|
|22 Jump Street (2...|Comedy|   597|       70.0|
|22 Jump Street (2...|Comedy|   155|       70.0|
|22 Jump Street (2...|Comedy|   115|       70.0|
|22 Jump Street (2...|Comedy|   436|       70.0|
|22 Jump Street (2...|Comedy|   406|       70.0|
|22 Jump Street (2...|Comedy|   412|       70.0|
|22 Jump Street (2...|Comedy|   587|       70.0|
|22 Jump Street (2...|Comedy|    44|       70.0|
|22 Jump Street (2...|Comedy|   236|       70.0|
|22 Jump Street (2...|Comedy|   350|       70.0|
|22 Jump Street (2..

tìm 5 phim rating cao nhất 

In [ ]:
window = Window.partitionBy(final_5['userId']).orderBy(final_5['sum(rating)'].desc())
final_5.select('*', f.rank().over(window).alias('rank')).filter(f.col('rank') <= 5)\
.groupBy('userId','genres').agg(f.collect_list('title').alias('top 5 title rated')).sort('userId').show(100, truncate=False)

+------+---------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|userId|genres   |top 5 title rated                                                                                                                                                                                                                                                                                           |
+------+---------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|1     |Action   |[Terminator 2: Judgmen

## **Câu 6:** For each user ID, show the two genres that received highest rating from that user, and the list of top 5 rated movies that belong to both genres and hasn’t been rated by that user. The results are shorted by the user ID.

In [41]:
userid_genres = movies.join(ratings, on=['movieId'], how='inner').withColumn('genres', f.split('genres','[|]')) \
.withColumn('genres', f.explode('genres')).groupBy('userId','genres').sum('rating').sort('userId')

In [42]:
userid_genres.show()

+------+---------+-----------+
|userId|   genres|sum(rating)|
+------+---------+-----------+
|     1|Animation|      136.0|
|     1|    Crime|      196.0|
|     1|  Fantasy|      202.0|
|     1|  Western|       30.0|
|     1| Children|      191.0|
|     1|  Romance|      112.0|
|     1|   Horror|       59.0|
|     1|      War|       99.0|
|     1|Film-Noir|        5.0|
|     1|Adventure|      373.0|
|     1|    Drama|      308.0|
|     1|   Action|      389.0|
|     1|  Musical|      103.0|
|     1|   Comedy|      355.0|
|     1|  Mystery|       75.0|
|     1|   Sci-Fi|      169.0|
|     1| Thriller|      228.0|
|     2|   Action|       43.5|
|     2|   Comedy|       28.0|
|     2|  Romance|        4.5|
+------+---------+-----------+
only showing top 20 rows



tìm top 1 , 2 thể loại có rating cao nhất

In [43]:
window = Window.partitionBy(userid_genres['userId']).orderBy(userid_genres['sum(rating)'].desc())
top_1_2 = userid_genres.select('*', f.rank().over(window).alias('rank')).filter(f.col('rank') <= 2)\
.select(f.col('userId'), f.col('genres'), f.col('sum(rating)')).withColumn('temp', f.lit('temp_'))\
.groupBy('userId').agg(f.collect_list('genres').alias('2 genres rating cao nhat'))

In [44]:
top_1_2.show()

+------+------------------------+
|userId|2 genres rating cao nhat|
+------+------------------------+
|     1|     [Action, Adventure]|
|     2|         [Drama, Action]|
|     3|        [Sci-Fi, Action]|
|     4|         [Drama, Comedy]|
|     5|         [Drama, Comedy]|
|     6|         [Drama, Comedy]|
|     7|     [Action, Adventure]|
|     8|         [Comedy, Drama]|
|     9|         [Drama, Comedy]|
|    10|       [Romance, Comedy]|
|    11|      [Action, Thriller]|
|    12|       [Romance, Comedy]|
|    13|    [Drama, Thriller,...|
|    14|        [Drama, Romance]|
|    15|         [Sci-Fi, Drama]|
|    16|       [Drama, Thriller]|
|    17|         [Drama, Action]|
|    18|         [Drama, Action]|
|    19|      [Comedy, Thriller]|
|    20|      [Children, Comedy]|
+------+------------------------+
only showing top 20 rows



In [45]:
top_12 = userid_genres.select('*', f.rank().over(window).alias('rank')).filter(f.col('rank') <= 2)\
.select(f.col('userId'), f.col('genres'), f.col('sum(rating)'))

In [46]:
top_12.show()

+------+---------+-----------+
|userId|   genres|sum(rating)|
+------+---------+-----------+
|     1|   Action|      389.0|
|     1|Adventure|      373.0|
|     2|    Drama|       66.0|
|     2|   Action|       43.5|
|     3|   Sci-Fi|       63.0|
|     3|   Action|       50.0|
|     4|    Drama|      418.0|
|     4|   Comedy|      365.0|
|     5|    Drama|       95.0|
|     5|   Comedy|       52.0|
|     6|    Drama|      506.0|
|     6|   Comedy|      428.0|
|     7|   Action|      208.5|
|     7|Adventure|      179.0|
|     8|   Comedy|       77.0|
|     8|    Drama|       72.0|
|     9|    Drama|       72.0|
|     9|   Comedy|       55.0|
|    10|  Romance|      260.0|
|    10|   Comedy|      258.0|
+------+---------+-----------+
only showing top 20 rows



Tìm các bộ phim theo rating (phim được user rating)

In [47]:
title_rated = movies.join(ratings, on=['movieId'], how='inner').withColumn('genres', f.split('genres','[|]'))\
.groupBy('userId').agg(f.collect_list('title').alias('title(rated)')).sort('userId')
#.groupBy('userId').agg(f.max('sum(rating)').alias('max of sum(rating)')).sort('userId').show()

In [48]:
title_rated.show(truncate=100)

+------+----------------------------------------------------------------------------------------------------+
|userId|                                                                                        title(rated)|
+------+----------------------------------------------------------------------------------------------------+
|     1|[Toy Story (1995), Grumpier Old Men (1995), Heat (1995), Seven (a.k.a. Se7en) (1995), Usual Suspe...|
|     2|[Shawshank Redemption, The (1994), Tommy Boy (1995), Good Will Hunting (1997), Gladiator (2000), ...|
|     3|[Dangerous Minds (1995), Schindler's List (1993), Courage Under Fire (1996), Operation Dumbo Drop...|
|     4|[Get Shorty (1995), Twelve Monkeys (a.k.a. 12 Monkeys) (1995), To Die For (1995), Seven (a.k.a. S...|
|     5|[Toy Story (1995), Get Shorty (1995), Babe (1995), Dead Man Walking (1995), Clueless (1995), Usua...|
|     6|[Jumanji (1995), Grumpier Old Men (1995), Waiting to Exhale (1995), Father of the Bride Part II (...|
|     7|[T

tìm tất cả phim theo từng genres rating cao nhất

In [49]:
title_all = movies.join(ratings, on=['movieId'], how='inner').withColumn('genres', f.split('genres','[|]'))\
.withColumn('genres', f.explode('genres')).groupBy('genres').agg(f.collect_set('title').alias('title all of'))

In [50]:
title_all.show(truncate=100)

+------------------+----------------------------------------------------------------------------------------------------+
|            genres|                                                                                        title all of|
+------------------+----------------------------------------------------------------------------------------------------+
|             Crime|[Stealing Rembrandt (Rembrandt) (2003), Girl with the Dragon Tattoo, The (Män som hatar kvinnor) ...|
|           Romance|[Vampire in Brooklyn (1995), Hysteria (2011), Far from Heaven (2002), Honey (2003), Ghost Town (2...|
|          Thriller|[Element of Crime, The (Forbrydelsens Element) (1984), 'Salem's Lot (2004), Transformers (2007), ...|
|         Adventure|[Ice Age: Collision Course (2016), Masters of the Universe (1987), Terminator Genisys (2015), Tit...|
|             Drama|[Airport '77 (1977), Meet John Doe (1941), 'Salem's Lot (2004), Element of Crime, The (Forbrydels...|
|               War|[Gen

join bảng

In [51]:
final_6 = title_rated.join(top_12, on=['userId'], how='inner').join(title_all, on=['genres'], how='inner')\
.withColumn('title', f.array_except('title all of','title(rated)')).sort('userId')\
.withColumn('title', f.explode('title'))\
.select(f.col('genres'), f.col('userId'), f.col('title'))\
.join(userid_title, on=['title'], how='inner')
#.groupBy('userId', 'title(rated)').agg(f.collect_list('genres')).show()
# .withColumn('title', f.array_except('title all of','title(rated)'))\
# .withColumn('title', f.explode('title')).sort('userId')\
# .select(f.col('genres'), f.col('userId'), f.col('title'))
#.join(userid_title, on=['userId', 'title'], how='inner')

In [52]:
final_6.show()

+--------------------+------+------+-----------+
|               title|genres|userId|sum(rating)|
+--------------------+------+------+-----------+
|22 Jump Street (2...|Action|     1|       70.0|
|22 Jump Street (2...|Action|     2|       70.0|
|22 Jump Street (2...|Action|     3|       70.0|
|22 Jump Street (2...|Comedy|     4|       70.0|
|22 Jump Street (2...|Comedy|     5|       70.0|
|22 Jump Street (2...|Comedy|     6|       70.0|
|22 Jump Street (2...|Action|     7|       70.0|
|22 Jump Street (2...|Comedy|     8|       70.0|
|22 Jump Street (2...|Comedy|     9|       70.0|
|22 Jump Street (2...|Comedy|    10|       70.0|
|22 Jump Street (2...|Action|    11|       70.0|
|22 Jump Street (2...|Comedy|    12|       70.0|
|22 Jump Street (2...|Action|    13|       70.0|
|22 Jump Street (2...|Action|    17|       70.0|
|22 Jump Street (2...|Action|    18|       70.0|
|22 Jump Street (2...|Comedy|    19|       70.0|
|22 Jump Street (2...|Comedy|    20|       70.0|
|22 Jump Street (2..

In [53]:
window = Window.partitionBy(final_6['userId']).orderBy(final_6['sum(rating)'].desc())
final_6.select('*', f.rank().over(window).alias('rank')).filter(f.col('rank') <= 5)\
.groupBy('userId').agg(f.collect_list('title').alias('top 5 title rated'),f.collect_set('genres').alias('genres')).sort('userId').show(100, truncate=False)

+------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------------+
|userId|top 5 title rated                                                                                                                                                                                                                       |genres                  |
+------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------------+
|1     |[Terminator 2: Judgment Day (1991), Lord of the Rings: The Fellowship of the Ring, The (2001), Apollo 13 (1995), Lord of the Rings: The Return of the King, The (2003), Lord of the Rings: The 

## **Câu 7:** 

## **Câu 8:** 